In [22]:
import math
def safe(n, k=2):
    return 2 * (n - 2**math.floor(math.log(n, 2))) + 1

In [26]:
safe(3004953)

1815603

## V2

First attempt: use a deque, and re-append skipped elfs to the end of the list (emulated circle). Append and pop is $\mathcal{O}(1)$, but need $n\times k$ of them, so solution is $\mathcal{O}(n\times k)$. Ok for $k \ll n$, but here we have $k = n/2$ which gives a quadratic solution. No good for massive input.

In [12]:
from collections import deque
def f(n, k):
    """k = constant"""
    elfs = deque(range(1, n+1))
    
    for steal_round in range(n-1):
        for skipped_elf in range(k-1):
            elfs.append(elfs.popleft())
        elfs.popleft()
    return elfs.popleft()

In [13]:
from collections import deque
def f(n):
    """k = n/2, does not handle odd/even correct, so does not work"""
    elfs = deque(range(1, n+1))
    
    for steal_round in range(n-1):
        k = (n-steal_round)//2
        print(elfs, n-steal_round, k)
        for skipped_elf in range(k):
            elfs.append(elfs.popleft())
        elfs.popleft()
    return elfs.popleft()

In [14]:
f(5)

deque([1, 2, 3, 4, 5]) 5 2
deque([4, 5, 1, 2]) 4 2
deque([2, 4, 5]) 3 1
deque([5, 2]) 2 1


5

## Working $\mathcal{O}(n)$ solution!

The idea: Set up elves in a linked list, and keep a pointer to both the stealer and the target. Remove the target, and advance the stealer one place. Also let the target skip over one elf in the case that the number of remaining elves are even. Also handle what happens in the end, if target and stealer becomes the same (stealer remains, target jumps to the other).

Removal with stored pointers are implemented $\mathcal{O}(1)$, and only $n-1$ of them are needed (the number of steals that happens). Still slow, as the input is very large, and the constant factor is considerable. But more than fast enough!

In [15]:
class Elf(object):
    def __init__(self, place):
        self.place = place
    def __str__(self):
        return str(self.place)
    


class ElfCircle(object):
    def __init__(self, n_elfs):
        head = Elf(1)
        elf = head
        for place in range(2, n_elfs+1):
            elf.left = Elf(place)
            elf.left.right = elf
            elf = elf.left
        elf.left = head
        head.right = elf
        
        self.head = head
        self.n_elfs = n_elfs
        
    def remove_elf(self, elf):
        if elf == self.head:
            self.head = elf.left
        elf.right.left = elf.left
        elf.left.right = elf.right
        self.n_elfs -= 1
        return elf.left
    
    def __len__(self):
        return self.n_elfs
    
    def __getitem__(self, i):  # Make it iterable!
        if not 0 <= i < len(self):
            raise IndexError(i)
        elf = self.head
        for _ in range(i):
            elf = elf.left
        return elf
    
    def __str__(self):
        return '->'.join(str(elf) for elf in self)  # Iterator is nice to have
        

In [17]:
n_elfs = 3004953
print('Seating all the elfs in a circle...', end='')
circle = ElfCircle(n_elfs)
print('Seated!\nStarting the stealing...')

stealer = circle[0]
target = circle[n_elfs//2]

for steal_round in range(n_elfs-1):
    target = circle.remove_elf(target)

    if len(circle) % 2 == 0:
        target = target.left
    stealer = stealer.left
    if stealer == target:
        target = target.left

print('The winning elf is number:', circle)

Seating all the elfs in a circle...Seated!
Starting the stealing...
The winning elf is number: 1410630
